In [48]:
import pandas as pd
import os
import re
from geopy import distance
#https://geopy.readthedocs.io/en/stable/#module-geopy.distance


In [49]:
airbnb_df = pd.read_csv('./Raw_Data/denver_listings.csv')
starbucks_df = pd.read_csv('./Raw_Data/starbucks.csv')
reviews_df = pd.read_csv('./Raw_Data/Denver_reviews.csv')

# AirBNB Listing Cleaning

In [1]:
display(airbnb_df.info())
display(airbnb_df.shape)
airbnb_columns = airbnb_df.columns.to_list()

NameError: name 'airbnb_df' is not defined

In [51]:
airbnb_columns_tokeep = \
['id',
 'description',
 'neighborhood_overview',
 'host_neighbourhood',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'price',
 'has_availability',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'reviews_per_month']


In [52]:
#Reviewed total of 75 original columns, eliminated unnecessary columns or columns 
#with excessively null data
airbnb_df = airbnb_df[airbnb_columns_tokeep]
# filled null values with -1 for easy identification - no organic data would contain -1
airbnb_df.fillna(-1,inplace=True)
# airbnb_df

In [53]:
display(airbnb_df.info())
# reviewed for columns with inappropriate data types, identified:
# bathrooms_text, price, first_review, last_review

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5388 entries, 0 to 5387
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              5388 non-null   int64  
 1   description                     5388 non-null   object 
 2   neighborhood_overview           5388 non-null   object 
 3   host_neighbourhood              5388 non-null   object 
 4   neighbourhood_cleansed          5388 non-null   object 
 5   latitude                        5388 non-null   float64
 6   longitude                       5388 non-null   float64
 7   property_type                   5388 non-null   object 
 8   room_type                       5388 non-null   object 
 9   accommodates                    5388 non-null   int64  
 10  bathrooms_text                  5388 non-null   object 
 11  bedrooms                        5388 non-null   float64
 12  beds                            53

None

In [54]:
#using regular expressions to extract the number of bathrooms
bathroom_re_format = r'(.*?)( .*)'
def extract_bathroom_count(string):
    string = str(string)
    re_return = re.match(bathroom_re_format, string)
    return float(re_return.group(1)) if re_return else None
airbnb_df['bathroom_count'] = airbnb_df['bathrooms_text'].apply(extract_bathroom_count)

#Using string methods to convert price to a numerical value
airbnb_df['price'] = airbnb_df['price'].str.replace(',','').str.strip('$')
airbnb_df['price'] = pd.to_numeric(airbnb_df['price'])

In [55]:
#Removing Price outliers (found to be likely glitches / faulty data)
airbnb_df = airbnb_df.loc[airbnb_df['price'] < 2001]
#Changing first and last reviews to datetimes
airbnb_df['first_review'] = pd.to_datetime(airbnb_df['first_review'], errors='coerce')
airbnb_df['last_review'] = pd.to_datetime(airbnb_df['last_review'], errors='coerce')

In [56]:
airbnb_df.sort_values(by='id',inplace=True)
airbnb_df.reset_index(drop=True, inplace=True)
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5367 entries, 0 to 5366
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              5367 non-null   int64         
 1   description                     5367 non-null   object        
 2   neighborhood_overview           5367 non-null   object        
 3   host_neighbourhood              5367 non-null   object        
 4   neighbourhood_cleansed          5367 non-null   object        
 5   latitude                        5367 non-null   float64       
 6   longitude                       5367 non-null   float64       
 7   property_type                   5367 non-null   object        
 8   room_type                       5367 non-null   object        
 9   accommodates                    5367 non-null   int64         
 10  bathrooms_text                  5367 non-null   object        
 11  bedr

In [57]:
airbnb_df

,id,description,neighborhood_overview,host_neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,bathroom_count
0,360,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,Highland,Highland,39.766415,-105.002098,Entire guesthouse,Entire home/apt,3,...,4.99,4.96,5.00,5.00,5.00,4.91,f,2,2.87,1.0
1,364,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",Five Points,Five Points,39.766720,-104.979060,Entire loft,Entire home/apt,3,...,4.78,4.81,4.95,4.96,4.65,4.71,f,1,0.50,1.5
2,590,"Large guest room in my home, where I also live...",I love the diversity of my neighborhood and it...,North Park Hill,North Park Hill,39.755110,-104.911090,Private room in home,Private room,3,...,4.78,4.58,4.93,4.95,4.76,4.85,f,2,4.04,1.0
3,592,This room is in the basement. It does not hav...,-1,North Park Hill,North Park Hill,39.754810,-104.911060,Private room in home,Private room,2,...,4.75,4.55,4.94,4.94,4.81,4.86,f,2,0.95,1.0
4,686,Thanks for visiting my Queen Bed Room site for...,"I love my Uptown neighborhood, which is within...",North Capitol Hill,North Capitol Hill,39.746950,-104.978380,Private room in home,Private room,2,...,4.75,4.80,4.86,4.91,4.87,4.81,f,2,1.39,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5362,985655038490529659,"Blake's house is a captivating haven, blending...",-1,Highland,Highland,39.765313,-105.013120,Entire home,Entire home/apt,14,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,1,-1.00,4.0
5363,985699485774546946,This is a fully restored property in one of th...,-1,Washington Park,Elyria Swansea,39.783094,-104.964334,Entire home,Entire home/apt,3,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,2,-1.00,1.0
5364,986480371259464482,Enjoy an industrial experience at this central...,"Nicknamed “RiNo,” the trendy River North Art D...",Five Points,Five Points,39.756581,-104.988746,Entire rental unit,Entire home/apt,2,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,f,1,-1.00,1.0
5365,986558663097275426,Beautiful/private one bedroom with private mod...,-1,South Denver,Rosedale,39.677558,-104.983539,Private room in home,Private room,2,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,t,1,-1.00,1.0


In [58]:
airbnb_longitude_mean = airbnb_df['longitude'].mean()
airbnb_latitude_mean = airbnb_df['latitude'].mean()
bnb_center_of_mass = (airbnb_latitude_mean,airbnb_longitude_mean)

# Airbnb Review Cleaning

In [59]:
display(reviews_df.head())
reviews_columns = reviews_df.columns.to_list()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,360,307152490,2018-08-13,46723582,Madeleine,"This space was perfect! Great location, hosts,..."
1,360,311601388,2018-08-21,68751664,Janelle,Hidden Gem in Denver. Magical Secret garden ce...
2,360,312497032,2018-08-23,57773484,Ziad,Amazing stay at one of the best kept AirBnB se...
3,360,313089933,2018-08-24,11982181,Andrea,The cottage has a great setting with a garden ...
4,360,314492150,2018-08-26,14620568,Iris,"Super cute place, very peaceful (loved the gar..."


In [60]:
#Sorting numerically by listing ID to consolidate reviews by listing
reviews_df.sort_values(by='listing_id',inplace=True)
#Select Relevant Columns
reviews_df = reviews_df[['listing_id','date','comments']]

In [61]:
reviews_df

,listing_id,date,comments
0,360,2018-08-13,"This space was perfect! Great location, hosts,..."
216,360,2020-02-09,素敵なホストファミリーのいる、アットホームで快適です◎
230,360,2020-02-11,Super nice and comfortable. Loved all the det...
231,360,2020-02-14,Enjoyed our stay. Great tips. Thanks!
232,360,2020-02-22,Beautiful little place and had everything we n...
...,...,...,...
305620,982032689667768275,2023-09-22,Best host ever
305622,982813615131924597,2023-09-21,"What a perfect space, location, and amenities...."
305623,982813615131924597,2023-09-22,Cute little suite by the lake had everything I...
305621,982813615131924597,2023-09-20,I recently had the pleasure of staying at this...


In [62]:
def count_mentions(string):
    string = str(string)
    string = string.lower()
    mentions = string.count('starbucks')
    return mentions

reviews_df['sb_mentions'] = reviews_df['comments'].apply(count_mentions)

In [63]:
aggregate_reviews = reviews_df.groupby('listing_id').sum().reset_index()
aggregate_reviews

,listing_id,date,comments,sb_mentions
0,360,2018-08-132020-02-092020-02-112020-02-142020-0...,"This space was perfect! Great location, hosts,...",0
1,364,2013-09-252011-07-172011-07-062011-06-262013-1...,We had a wonderful stay at Jason's loft on our...,0
2,590,2016-11-062016-11-132016-11-202017-04-142016-1...,Jill made me feel very welcomed. It felt like ...,0
3,592,2010-10-192010-10-272010-11-182011-01-072016-0...,"My mother, Mary, and I experienced our first a...",0
4,686,2017-07-072012-03-042012-02-132016-11-032016-1...,Great place to stay while visiting Denver. Wha...,0
...,...,...,...,...
4598,978569724187627478,2023-09-152023-09-202023-09-21,Clean comfortable rooms. Tastefully decorated ...,0
4599,979137158360962181,2023-09-22,"amazing host,everything is clean!",0
4600,979571767218706397,2023-09-17,Nice and cozy place to stay for a weekend trip...,0
4601,982032689667768275,2023-09-202023-09-22,It was very easy to get in and out of the hous...,0


# Starbucks Data Cleaning

In [64]:
display(starbucks_df.head())
starbucks_columns = starbucks_df.columns.to_list()

,Unnamed: 0,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,7958,75988-107245,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 4:0...",anc-alaska-ticketing-door-3-5000-west-internat...,61.174006,-149.981584,5000 West International Airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,city,NaN
1,7959,75697-94324,US,LS,"[{'dayName': 'Today', 'hours': '3:30 AM to 12:...",anc-concourse-c-gate-c-1-5000-w-intl-airport-r...,61.173768,-149.982953,5000 W Intl Airport Rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,London,187.0
2,7960,74430-51979,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",carrs-anchorage-1805-1650-w-northern-lights-bl...,61.194274,-149.915872,1650 W Northern Lights Blvd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,New York,187.0
3,7961,79664-81037,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-anchorage-1812-1725-abbot-rd-anchorage...,61.137251,-149.956031,1725 Abbot Rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Chicago,178.0
4,7962,25371-240771,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 8:0...",benson-minnesota-1515-w-29th-ave-anchorage-ak-...,61.194103,-149.912474,1515 W 29th Ave,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Las Vegas,165.0


In [65]:
starbucks_columns
# Initial removal of un-used columns (address, store number, etc.)
starbucks_columns_tokeep = \
['Unnamed: 0',
 'countryCode',
 'ownershipTypeCode',
 'latitude',
 'longitude',
 'city',
 'countrySubdivisionCode',
 'postalCode']
starbucks_df = starbucks_df[starbucks_columns_tokeep]
starbucks_df.rename(columns={'Unnamed: 0':'store_id'},inplace=True)

In [66]:
#Filter to US based Colorado stores (searching for denver suburbs possibly included therein)
starbucks_df = starbucks_df.loc[(starbucks_df['countryCode'] == 'US')
                                & (starbucks_df['countrySubdivisionCode'] == 'CO')]

In [67]:
#Truncating zip+4 postal codes to 5 digit codes
starbucks_df['postalCode'] = starbucks_df['postalCode'].str[:5]

In [68]:
#calculate the distance to denver from each starbucks, populate dataframe row
def distance_to_bnb_center(row):
    lat = row['latitude']
    long = row['longitude']
    radius = round(distance.distance((lat,long),bnb_center_of_mass).miles,2)
    return radius

starbucks_df['distance_to_den'] = starbucks_df.apply(distance_to_bnb_center, axis=1)

In [69]:
#eliminating starbucks outside a 25 mile radius from the airbnb center
starbucks_df = starbucks_df.loc[starbucks_df['distance_to_den'] < 25]
#re-index after filters
starbucks_df.reset_index(inplace=True)

In [70]:
starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   303 non-null    int64  
 1   store_id                303 non-null    int64  
 2   countryCode             303 non-null    object 
 3   ownershipTypeCode       303 non-null    object 
 4   latitude                303 non-null    float64
 5   longitude               303 non-null    float64
 6   city                    303 non-null    object 
 7   countrySubdivisionCode  303 non-null    object 
 8   postalCode              303 non-null    object 
 9   distance_to_den         303 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 23.8+ KB


# Early Analytics

In [71]:
#Short check to find how many listings do not have at least one review
listings_ids = list(airbnb_df['id'].unique())
reviews_ids = list(reviews_df['listing_id'].unique())
print(len(listings_ids)-len(reviews_ids))

764


In [72]:
#Given 303 starbucks locations and 5367 airbnb locations, calculate a distance from each airbnb to each starbucks
#create two list, store them in each dataframe 
def starbucks_distance_calc(airbnb):
    """ Receives: a row from an airbnb listings dataframe 
    returns: a dictionary with the distance to every starbucks"""
    lat = airbnb['latitude']
    long = airbnb['longitude']
    bnb_location = (lat, long)
    distance_list = []
    for index, row in starbucks_df.iterrows():
        sb_lat = row['latitude']
        sb_long = row['longitude']
        sb_location = (sb_lat, sb_long)
        sb_id = row['store_id']
        radius = round(distance.distance(bnb_location,sb_location).miles,2)
        distance_list.append(radius)
    return distance_list
#Create the list of distances to every starbucks, applied to the airbnb_df
airbnb_df['starbucks_distances'] = airbnb_df.apply(starbucks_distance_calc, axis=1)

In [73]:
#Airbnb Distance calc (opposite of starbucks above)
def airbnb_distance_calc(starbucks):
    """ Receives: a row from an airbnb listings dataframe 
    returns: a dictionary with the distance to every starbucks"""
    lat = starbucks['latitude']
    long = starbucks['longitude']
    starbucks_location = (lat, long)
    distance_list = []
    for index, row in airbnb_df.iterrows():
        bnb_lat = row['latitude']
        bnb_long = row['longitude']
        bnb_location = (bnb_lat, bnb_long)
        bnb_id = row['id']
        radius = round(distance.distance(bnb_location,starbucks_location).miles,2)
        distance_list.append(radius)
    return distance_list
#Create the list of distances to every starbucks, applied to the airbnb_df
starbucks_df['airbnb_distances'] = starbucks_df.apply(airbnb_distance_calc, axis=1)

In [74]:
def mentions_lookup(row):
    bnb_id = row['id']
    try:
        mentions = aggregate_reviews.loc[aggregate_reviews['listing_id']==bnb_id,'sb_mentions'].item()
    except:
        mentions = 0
    return mentions
airbnb_df['starbucks_review_mentions'] = airbnb_df.apply(mentions_lookup, axis=1)

In [75]:
airbnb_df.set_index('id', inplace=True)
starbucks_df.set_index('store_id', inplace=True)
airbnb_df.to_csv('./Clean_Data/AirBNB_Data.csv', index=True)
starbucks_df.to_csv('./Clean_Data/Starbucks_Data.csv', index=True)

In [92]:
import ast
test_read_sb = pd.read_csv('./Clean_Data/Starbucks_Data.csv')
test_read_sb['airbnb_distances'] = test_read_sb['airbnb_distances'].apply(ast.literal_eval)

In [93]:
test_read_sb

,store_id,index,countryCode,ownershipTypeCode,latitude,longitude,city,countrySubdivisionCode,postalCode,distance_to_den,airbnb_distances
0,1743,3739,US,CO,39.652780,-104.928197,Denver,CO,80222,6.65,"[8.77, 8.31, 7.12, 7.1, 7.03, 5.96, 6.87, 9.22..."
1,1744,3740,US,CO,39.653634,-104.915687,Denver,CO,80222,6.89,"[9.04, 8.5, 7.01, 6.98, 7.25, 6.35, 7.1, 9.58,..."
2,1745,3741,US,LS,39.651856,-104.914206,Denver,CO,80222,7.03,"[9.19, 8.65, 7.13, 7.1, 7.4, 6.49, 7.24, 9.73,..."
3,1746,3742,US,LS,39.666547,-104.937194,Denver,CO,80222,5.59,"[7.71, 7.26, 6.27, 6.25, 5.97, 4.92, 5.81, 8.1..."
4,1747,3743,US,CO,39.667560,-104.939130,Denver,CO,80210,5.49,"[7.6, 7.16, 6.22, 6.2, 5.86, 4.8, 5.71, 8.06, ..."
...,...,...,...,...,...,...,...,...,...,...,...
298,16058,4142,US,LS,39.415198,-104.879530,Castle Rock,CO,80108,23.11,"[25.1, 24.83, 23.51, 23.49, 23.49, 21.98, 23.3..."
299,16059,4143,US,CO,39.423908,-104.879847,Castle Rock,CO,80108,22.52,"[24.51, 24.24, 22.91, 22.89, 22.9, 21.4, 22.75..."
300,16063,4147,US,LS,39.468564,-104.879455,Castle Rocks,CO,80108,19.54,"[21.57, 21.24, 19.84, 19.82, 19.92, 18.49, 19...."
301,16064,4148,US,CO,39.471811,-104.876571,Castle Rock,CO,80108,19.37,"[21.4, 21.07, 19.63, 19.61, 19.74, 18.33, 19.5..."
